In [3]:
#install dataset from HF and store it on pandas dataframe 
#!pip install datasets 

from datasets import load_dataset

#the dataset is downloaded from hugging face
#Note: When experimenting on other datasets, it needs to be arranged in this manner 
#https://huggingface.co/datasets/amazon_reviews_multi

train_ds = load_dataset("amazon_reviews_multi", "en", split='train')
val_ds = load_dataset("amazon_reviews_multi", "en", split='validation')
test_ds = load_dataset("amazon_reviews_multi", "en", split='test')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to C:/Users/Mir Info/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


Found cached dataset amazon_reviews_multi (C:/Users/Mir Info/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)
Found cached dataset amazon_reviews_multi (C:/Users/Mir Info/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


In [17]:
#store as pandas dataframe 
import pandas as pd
df_train = pd.DataFrame(train_ds)
df_val = pd.DataFrame(val_ds)
df_test = pd.DataFrame(test_ds)

In [19]:
#filtering the dataset by cutting off reviews and titles that are too small 

cutoff_title = 5
cutoff_body = 20

In [20]:
df_train = df_train[(df_train['review_title'].apply(lambda x: len(x.split()) >= cutoff_title)) & (df_train['review_body'].apply(lambda x: len(x.split()) >= cutoff_body))]
df_val = df_val[(df_val['review_title'].apply(lambda x: len(x.split()) >= cutoff_title)) & (df_val['review_body'].apply(lambda x: len(x.split()) >= cutoff_body))]
df_test = df_test[(df_test['review_title'].apply(lambda x: len(x.split()) >= cutoff_title)) & (df_test['review_body'].apply(lambda x: len(x.split()) >= cutoff_body))]

In [25]:
#limit the size of the dataset for faster training 
print(len(df_train), len(df_val), len(df_test))

56287 1428 1403


In [32]:
df_train = df_train.sample(20000, random_state=42)
df_train = df_train.rename(columns={"review_body": "text", "review_title": "summary"})

df_val = df_val.sample(1000, random_state=42)
df_val = df_val.rename(columns={"review_body": "text", "review_title": "summary"})

df_test = df_test.sample(1000, random_state=42)
df_test = df_test.rename(columns={"review_body": "text", "review_title": "summary"})

In [30]:
print(df_train['text'][0])

Arrived broken. Manufacturer defect. Two of the legs of the base were not completely formed, so there was no way to insert the casters. I unpackaged the entire chair and hardware before noticing this. So, I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review of part of a chair I never got to sit in. I will go so far as to include a picture of what their injection molding and quality assurance process missed though. I will be hesitant to buy again. It makes me wonder if there aren't missing structures and supports that don't impede the assembly process.


In [38]:
df_train["summary"][0]

"I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review ..."

In [40]:
# saving the data as CSV file and upload to S3

df_train.to_csv('C:/Users/Mir Info/Projects/speech_to_text/train.csv', index=False, columns=['text', 'summary'])
df_val.to_csv('C:/Users/Mir Info/Projects/speech_to_text/val.csv', index=False, columns=['text', 'summary'])
df_test.to_csv('C:/Users/Mir Info/Projects/speech_to_text/test.csv', index=False, columns=['text', 'summary'])

##### Do not run this cell code if you do not have an AWS server for model testing

In [46]:
import boto3
import sagemaker

In [52]:
session = boto3.session.Session(region_name='us-west-2')
sm_client = session.client('sagemaker')

In [53]:
#I do not have a AWS account, for the moment the model dataset and model training will be done locally 
!aws s3 cp C:/Users/Mir Info/Projects/speech_to_text/train.csv s3://<destination>/train.csv
!aws s3 cp C:/Users/Mir Info/Projects/speech_to_text/val.csv s3://<destination>/val.csv
!aws s3 cp C:/Users/Mir Info/Projects/speech_to_text/test.csv s3://<destination>/test.csv


'aws' is not recognized as an internal or external command,
operable program or batch file.
'aws' is not recognized as an internal or external command,
operable program or batch file.
'aws' is not recognized as an internal or external command,
operable program or batch file.


----

#### Creating a baseline score

In [54]:
!pip install "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 2.2/2.2 MB 485.9 kB/s eta 0:00:00
     ------------------------------------ 221.8/221.8 kB 564.1 kB/s eta 0:00:00
     ------------------------------------ 880.6/880.6 kB 497.4 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 2.0/2.0 MB 489.5 kB/s eta 0:00:00
     -------------------------------------- 69.8/69.8 kB 476.3 kB/s eta 0:00:00
     ------------------------------------ 130.2/130.2 kB 641.5 kB/s eta 0:00:00
     ---------------------------------------- 7.2/7.2 MB 220.0 kB/s eta 0:00:00
     -------------------------------------- 73.4/73.4 kB 579.2 kB/s eta 0:00:00
     -------------------------------------- 72.7/72.7 kB 568.0 kB/s eta 0:00:00
     ------------------------------------ 154.0/154.0 kB 297.0 kB/s eta 0:00:00
INFO: pip is looking

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.
evaluate 0.4.0 requires datasets>=2.0.0, but you have datasets 1.6.2 which is incompatible.
evaluate 0.4.0 requires huggingface-hub>=0.7.0, but you have huggingface-hub 0.0.8 

In [57]:
from datasets import load_metric
metric = load_metric("rouge")

C:\Users\Mir Info\AppData\Local\Temp\ipykernel_9644\2721381364.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [58]:
#The function is taken from the notebook 
#https://github.com/huggingface/transformers/blob/v4.6.1/examples/pytorch/summarization/run_summarization.py

def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result 

In [64]:
ref_summaries = list(df_test['summary'])

In [66]:
#comparing the reference texts with the first sentence 3 summaries 

import re
for i in range (3):
    candidate_summaries = list(df_test['text'].apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:i+1])))
    print(f"First {i+1} senctences: Scores {calc_rouge_scores(candidate_summaries, ref_summaries)}")

First 1 senctences: Scores {'rouge1': 27.1, 'rouge2': 17.5, 'rougeL': 25.7, 'rougeLsum': 25.7}
First 2 senctences: Scores {'rouge1': 26.7, 'rouge2': 17.2, 'rougeL': 24.6, 'rougeLsum': 24.6}
First 3 senctences: Scores {'rouge1': 24.6, 'rouge2': 15.9, 'rougeL': 22.7, 'rougeLsum': 22.6}


rouge 1 is a better score 